# Phase 2 - YOLO object detection

## Imports

In [1]:
import numpy as np
import time
import cv2 
import os
import glob
import matplotlib.pyplot as plt
COLOR = (170,205,102)
COLOR_TEXT = (255,0,255)

### Reading image

In [2]:
images_paths = glob.glob("Downloads/project_data/test_images/*.jpg")
images = [plt.imread(image) for image in images_paths]

# img = cv2.imread(r"Downloads/test5.jpg")
img = images[7]
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

(H, W) = img.shape[:2]


## Passing weights and configuration file to neural network

In [3]:
weights_path = os.path.join("Downloads\yolo", "yolov3.weights")
config_path = os.path.join("Downloads\yolo", "yolov3.cfg")

In [4]:
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)

## Getting output layers

In [5]:
names = net.getLayerNames()
# extracting the layer names
layer_names = [ names[ i[0]-1 ] for i in net.getUnconnectedOutLayers() ]

## Inserting image blobs as input to neural network

In [6]:
blob = cv2.dnn.blobFromImage(img, 1/255.0, (416,416), crop=False, swapRB = False) # swapRB = False lw ana swapthom fo2 w ana bimport

In [7]:
net.setInput(blob)

# Calculating the runtime for NN
start_time = time.time()

layers_output = net.forward(layer_names) # performing a forward pass through the layers
print("Runtime for Detecting Objects is: {}".format(time.time() - start_time))

Runtime for Detecting Objects is: 1.384394884109497


In [8]:
boxes = []
classIDs = []
confidences = []

for output in layers_output:
    print("hello")
    for detection in output:

        scores = detection[5:]
        classID = np.argmax(scores)
        
        confidence = scores[classID]

        if (confidence > 0.85):
            box = detection[:4]*np.array([W, H, W, H]) # leh 3mlna multiply????
            bx, by, bw, bh = box.astype("int")

            # 3ayzeen el upperleft corners msh el centers 3shan kdh 3mlna el subtraction dh
            x = int(bx - bw/2)
            y = int(by - bh/2)

            boxes.append([x, y, int(bw), int(bh)])
            confidences.append(float(confidence))
            classIDs.append(classID)



hello
hello
hello


## Performing Non-Maximum Suppression to get rid of overlapping bounding boxes

In [9]:
idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.8,0.8) # lazem confidences tb2a float

In [10]:
label_path = os.path.join("Downloads/yolo", "coco.names")
labels = open(label_path).read().strip().split("\n") # strip btremove el spaces ely mwgoda

In [11]:
for i in idxs.flatten():
    (x, y, w, h) = [boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]]

    # drawing the boxes
    img = cv2.rectangle(img, (x,y), (x+w,y+h), COLOR, 2 )

    # Putting name of detected object and accuracy of detection
    cv2.putText(img,"{} : {:.3f}".format(labels[classIDs[i]], confidences[i]), (x,y-5), cv2.FONT_HERSHEY_SIMPLEX,0.5, COLOR_TEXT,2)
    

In [12]:
cv2.imshow("Final Image",img)
cv2.waitKey(0)
cv2.destroyAllWindows() 
# plt.imshow(img)